In [13]:
import xlrd
import agate




In [11]:
workbook = xlrd.open_workbook('unicef_oct_2014.xls')
workbook.nsheets
workbook.sheet_names()

[u'Child labour  ']

In [12]:
sheet = workbook.sheets()[0]
sheet.nrows
sheet.row_values(0)

for r in range(sheet.nrows):
    print r, sheet.row(r)

0 [text:u'UNICEF global databases', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'']
1 [text:u'Child labour', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'']
2 [text:u'Updated November 2014', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'']
3 [empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'', empty:u'']
4 [empty:u'', text:u'Total (%)', empty:u'', text:u'Sex (%)', empty:u'', text:u'Place of residence (%)', empty:u'', text:u'Household wealth quintile (%)', empty:u'', empty:u'', empty:u'', empty:u'', text:u'Reference Year', text:u'Data Source']
5 [text:u'Countries and areas', empty:u'', empty:u'', text:u'Male', text:u'F

In [14]:
title_rows = zip(sheet.row_values(4), sheet.row_values(5))
title_rows

[(u'', u'Countries and areas'),
 (u'Total (%)', u''),
 (u'', u''),
 (u'Sex (%)', u'Male'),
 (u'', u'Female'),
 (u'Place of residence (%)', u'Urban'),
 (u'', u'Rural'),
 (u'Household wealth quintile (%)', u'Poorest'),
 (u'', u'Second'),
 (u'', u'Middle'),
 (u'', u'Fourth'),
 (u'', u'Richest'),
 (u'Reference Year', u''),
 (u'Data Source', u'')]

In [16]:
titles = [t[0] + ' ' + t[1] for t in title_rows]
titles = [t.strip() for t in titles]
titles

[u'Countries and areas',
 u'Total (%)',
 u'',
 u'Sex (%) Male',
 u'Female',
 u'Place of residence (%) Urban',
 u'Rural',
 u'Household wealth quintile (%) Poorest',
 u'Second',
 u'Middle',
 u'Fourth',
 u'Richest',
 u'Reference Year',
 u'Data Source']

In [17]:
from xlrd.sheet import ctype_text
import agate

text_type = agate.Text()
number_type = agate.Number()
boolean_type = agate.Boolean()
date_type = agate.Date()

example_row = sheet.row(6)
print example_row
print example_row[0].ctype
print example_row[0].value
print ctype_text


[text:u'Afghanistan', number:10.3, empty:u'', number:11.0, number:9.6, number:7.9, number:10.8, number:11.9, number:11.8, number:10.1, number:9.7, number:7.8, text:u'2010-2011', text:u'MICS']
1
Afghanistan
{0: 'empty', 1: 'text', 2: 'number', 3: 'xldate', 4: 'bool', 5: 'error', 6: 'blank'}


In [19]:
types = []

for v in example_row:
    value_type = ctype_text[v.ctype]
    if value_type == 'text':
        types.append(text_type)
    elif value_type == 'number':
        types.append(number_type)
    elif value_type == 'xldate':
        types.append(date_type)
    else:
        types.append(text_type)
types
titles

[u'Countries and areas',
 u'Total (%)',
 u'',
 u'Sex (%) Male',
 u'Female',
 u'Place of residence (%) Urban',
 u'Rural',
 u'Household wealth quintile (%) Poorest',
 u'Second',
 u'Middle',
 u'Fourth',
 u'Richest',
 u'Reference Year',
 u'Data Source']

In [20]:
table = agate.Table(country_rows, titles, types)

C:\Users\Spencer\Anaconda3\envs\py27\lib\site-packages\agate\utils.py:276: UnnamedColumnWarning: Column 2 has no name. Using "c".


CastError: Can not parse value "" as Decimal. Error at row 1 column Place of residence (%) Urban.

In [21]:
def remove_bad_chars(val):
    if val == '-':
        return None
    return val

cleaned_rows = []
for row in country_rows:
    cleaned_row = [remove_bad_chars(rv) for rv in row]
    cleaned_rows.append(cleaned_row)

table = agate.Table(cleaned_rows, titles, types)
table


def get_new_array(old_array, function_to_clean):
    new_arr = []
    for row in old_array:
        cleaned_row = [function_to_clean(rv) for rv in row]
        new_arr.append(cleaned_row)
    return new_arr

cleaned_rows = get_new_array(country_rows, remove_bad_chars)

table = agate.Table(cleaned_rows, titles, types)
table.print_table(max_columns=7)
table.column_names

| Countries and areas  | Total (%) | c | Sex (%) Male | Female | Place of residenc... | Rural | ... |
| -------------------- | --------- | - | ------------ | ------ | -------------------- | ----- | --- |
| Afghanistan          |      10.3 |   |         11.0 |    9.6 |                  7.9 |  10.8 | ... |
| Albania              |       5.1 | y |          6.2 |    3.9 |                      |       | ... |
| Algeria              |       4.7 | y |          5.5 |    3.9 |                  3.9 |   5.6 | ... |
| Angola               |      23.5 |   |         22.1 |   24.8 |                      |       | ... |
| Argentina            |       4.4 |   |          4.8 |    3.9 |                      |       | ... |
| Armenia              |       3.9 |   |          4.7 |    2.9 |                  2.4 |   6.0 | ... |
| Azerbaijan           |       6.5 | y |          7.5 |    5.4 |                  2.1 |  10.7 | ... |
| Bahrain              |       4.6 |   |          6.3 |    3.0 |                  

C:\Users\Spencer\Anaconda3\envs\py27\lib\site-packages\agate\utils.py:276: UnnamedColumnWarning: Column 2 has no name. Using "c".
C:\Users\Spencer\Anaconda3\envs\py27\lib\site-packages\agate\utils.py:276: UnnamedColumnWarning: Column 2 has no name. Using "c".


(u'Countries and areas',
 u'Total (%)',
 'c',
 u'Sex (%) Male',
 u'Female',
 u'Place of residence (%) Urban',
 u'Rural',
 u'Household wealth quintile (%) Poorest',
 u'Second',
 u'Middle',
 u'Fourth',
 u'Richest',
 u'Reference Year',
 u'Data Source')

In [25]:
most_egregious = table.order_by('Total (%)', reverse=True).limit(10)
for r in most_egregious.rows:
    print r

<agate.Row: (u'Somalia', Decimal('49.0'), None, Decimal('44.5'), Decimal('53.6'), ...)>
<agate.Row: (u'Cameroon', Decimal('41.7'), None, Decimal('43.1'), Decimal('40.2'), ...)>
<agate.Row: (u'Zambia', Decimal('40.6'), u'y', Decimal('41.6'), Decimal('39.5'), ...)>
<agate.Row: (u'Burkina Faso', Decimal('39.2'), None, Decimal('42.3'), Decimal('36.0'), ...)>
<agate.Row: (u'Guinea-Bissau', Decimal('38.0'), None, Decimal('39.5'), Decimal('36.4'), ...)>
<agate.Row: (u'Ghana', Decimal('33.9'), None, Decimal('33.8'), Decimal('34.0'), ...)>
<agate.Row: (u'Nepal', Decimal('33.9'), u'y', Decimal('30.2'), Decimal('37.8'), ...)>
<agate.Row: (u'Peru', Decimal('33.5'), u'y', Decimal('30.6'), Decimal('36.3'), ...)>
<agate.Row: (u'Niger', Decimal('30.5'), None, Decimal('30.8'), Decimal('30.1'), ...)>
<agate.Row: (u'Central African Republic', Decimal('28.5'), None, Decimal('27.2'), Decimal('29.9'), ...)>


In [26]:
most_females = table.order_by('Female', reverse=True).limit(10)
for r in most_females.rows:
    print '{}: {}%'.format(r['Countries and areas'], r['Female'])

Cabo Verde: None%
Chile: None%
Ecuador: None%
Somalia: 53.6%
Cameroon: 40.2%
Zambia: 39.5%
Nepal: 37.8%
Guinea-Bissau: 36.4%
Peru: 36.3%
Burkina Faso: 36.0%


In [27]:
female_data = table.where(lambda r: r['Female'] is not None)
most_females = female_data.order_by('Female', reverse=True).limit(10)
for r in most_females.rows:
    print '{}: {}%'.format(r['Countries and areas'], r['Female'])

Somalia: 53.6%
Cameroon: 40.2%
Zambia: 39.5%
Nepal: 37.8%
Guinea-Bissau: 36.4%
Peru: 36.3%
Burkina Faso: 36.0%
Ghana: 34.0%
Rwanda: 30.4%
Niger: 30.1%


In [28]:
(lambda x: 'Positive' if x >= 1 else 'Zero or Negative')(0)
(lambda x: 'Positive' if x >= 1 else 'Zero or Negative')(4)

'Positive'

In [29]:
table.columns['Place of residence (%) Urban'].aggregate(agate.Mean())
col = table.columns['Place of residence (%) Urban']
table.aggregate(agate.Mean('Place of residence (%) Urban'))

AttributeError: 'Column' object has no attribute 'aggregate'

In [30]:
has_por = table.where(lambda r: r['Place of residence (%) Urban'] is not None)
has_por.aggregate(agate.Mean('Place of residence (%) Urban'))

Decimal('10.41204819277108433734939759')

In [31]:
first_match = has_por.find(lambda x: x['Rural'] > 50)
first_match['Countries and areas']


u'Bolivia (Plurinational State of)'

In [32]:
ranked = table.compute([('Total Child Labor Rank', agate.Rank('Total (%)', reverse=True)),])
for row in ranked.order_by('Total (%)', reverse=True).limit(20).rows:
    print row['Total (%)'], row['Total Child Labor Rank']

49.0 1
41.7 2
40.6 3
39.2 4
38.0 5
33.9 6
33.9 6
33.5 8
30.5 9
28.5 10
28.5 10
28.3 12
28.3 12
27.8 14
27.6 15
27.4 16
26.4 17
26.4 17
26.3 19
26.1 20


In [33]:
def reverse_percent(row):
    return 100 - row['Total (%)']
ranked = table.compute([('Children not working (%)',
                             agate.Formula(number_type, reverse_percent)),
                            ])

In [34]:
ranked = ranked.compute([('Total Child Labor Rank',
                              agate.Rank('Children not working (%)')),
                           ])

In [35]:
for row in ranked.order_by('Total (%)', reverse=True).limit(20).rows:
    print row['Total (%)'], row['Total Child Labor Rank']

49.0 1
41.7 2
40.6 3
39.2 4
38.0 5
33.9 6
33.9 6
33.5 8
30.5 9
28.5 10
28.5 10
28.3 12
28.3 12
27.8 14
27.6 15
27.4 16
26.4 17
26.4 17
26.3 19
26.1 20


In [ ]:
import matplotlib.pyplot as plt


plt.plot(africa_cpi_cl.columns['CPI 2013 Score'],
         africa_cpi_cl.columns['Total (%)'])
plt.xlabel('CPI Score - 2013')
plt.ylabel('Child Labor Percentage')
plt.title('CPI & Child Labor Correlation')
plt.show()


plt.plot(highest_cpi_cl.columns['CPI 2013 Score'],
         highest_cpi_cl.columns['Total (%)'])
plt.xlabel('CPI Score - 2013')
plt.ylabel('Child Labor Percentage')
plt.title('CPI & Child Labor Correlation')
plt.show()